In [1]:
#!pip install opencv-python

import cv2

import numpy as np
from PIL import Image, ImageDraw
import cv2
import sys
import numpy as np
import argparse
import pickle
import os
from skimage import io
import matplotlib.pyplot as plt

In [ ]:
from flask import Flask, Response
from kafka import KafkaConsumer
import numpy as np

#!pip install neo4j
from time import sleep
from json import loads
import json
from datetime import datetime
import pandas as pd
from neo4j import GraphDatabase

consumer = KafkaConsumer('testOntology', bootstrap_servers='localhost:9092')

readed = 0

for message in consumer:
    k = message.value
    while True:
        try:
            if(k.decode() == 'start video'):
                readed = 1
                print(str(k.decode()))
            elif(k.decode() == 'end video'):
                readed = 1
                print(str(k.decode()))
            break
        except:
            break
    if(readed == 1):
        readed = 0
        continue

    y = np.frombuffer(k, dtype='uint8')
    img = y.reshape(360, 640, 3)
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()

    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    objects = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            print(label)
            objects.append(label)

    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "1234"))
    session = data_base_connection.session()
    if(objects.count("person") >=1 and objects.count("knife") >=1):
        s="merge (v1:Video {name:'video1'})"+\
          "merge (e1:Event {name:'event1'})"+\
          "merge (p1:Person {name: 'person1'})"+\
          "merge (p2:Person {name: 'person2'})"+\
          "merge (k:Knife {name: 'knife'})"+\
          "merge (i:Indicator {name: 'Murder'})"+\
          "MERGE (e1)-[:OBTAINED_FROM]->(v1)"+\
          "MERGE (e1)-[:ASSOCIATED_WITH]->(i)"+\
          "MERGE (e1)-[:HAS_OBJECTS]->(p1)"+\
          "MERGE (e1)-[:HAS_OBJECTS]->(p2)"+\
          "MERGE (e1)-[:HAS_OBJECTS]->(k)"

        session.run(s)

start video
knife
knife
knife
knife
knife
tie
end video
start video
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
end video
start video
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person
knife
person


In [ ]:
objects

In [ ]:
if(objects.count("person") >=2):
    print("ok")

In [ ]:
!pip install neo4j
from kafka import KafkaConsumer
from time import sleep
from json import loads
import json
from datetime import datetime
import pandas as pd
from neo4j import GraphDatabase

In [ ]:
data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "1234"))
session = data_base_connection.session() 

In [ ]:
if(objects.count("person") >=1 and objects.count("knife") >=1):
    s="merge (v1:Video {name:'video1'})"+\
      "merge (e1:Event {name:'event1'})"+\
      "merge (p1:Person {name: 'person1'})"+\
      "merge (p2:Person {name: 'person2'})"+\
      "merge (k:Knife {name: 'knife'})"+\
      "merge (i:Indicator {name: 'Murder'})"+\
      "MERGE (e1)-[:OBTAINED_FROM]->(v1)"+\
      "MERGE (e1)-[:ASSOCIATED_WITH]->(i)"+\
      "MERGE (e1)-[:HAS_OBJECTS]->(p1)"+\
      "MERGE (e1)-[:HAS_OBJECTS]->(p2)"+\
      "MERGE (e1)-[:HAS_OBJECTS]->(k)"

session.run(s)

In [ ]:
#"MERGE (y)-[:OBTAINED_FROM]->(x)"+\
#"MERGE (y)-[:ASSOCIATED_WITH]->(i)"+\
#"MERGE (y)-[:HAS_OBJECTS]->(z1)"+\
#"MERGE (y)-[:HAS_OBJECTS]->(z2)"+\
#"MERGE (y)-[:HAS_OBJECTS]->(w)"+\
#"MERGE (y)-[:HAS_ACTIONS]->(s)"+\
#"MERGE (s)-[:MADE_BY]->(z1)"+\
#"MERGE (s)-[:MADE_ON]->(z2)"+\